### Auto Approval Cron Job

1. Settings
    1. account ID : accID
    1. vehicle ID : vehID
    1. region
    1. Time range: timeRange
1. Get relevant alerts
    1. Fetch by {accID, timeRange} -- via API
    1. Filter alerts that have remarks/approval 
    1. Filter alert by vehID
1. For each alert get video from aws -- parallel thread
1. Eval for approval <bf>evalApproval(localVideoFile, alertType)</bf>
1. Log results:
    1. [alert_id, image, type, result, stat, run_ts, run_times]
    1. 12345678, <image>, seatbelt, approve, 867/1000, 12345678, (1, 1.3)
    1. 12345678, <image>, mobile, ignore, 67/300, 12345679, (1, 1.3)
1. Update alerts via API


In [78]:
import json, glob, os, sys
from datetime import datetime
import time
import utilsAWS as uAws;
import intanglesAPIs as iApis
import imageio
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
reload(uAws)
uAws.gVerb = False

from PIL import Image
import io
import base64

def getBase64Image(img):
    # Convert NumPy array to PIL Image
    pil_image = Image.fromarray(img)

    # Save the PIL Image to a BytesIO buffer in PNG format
    buffer = io.BytesIO()
    pil_image.save(buffer, format="PNG")
    image_bytes = buffer.getvalue()

    # Base64 encode the image bytes
    base64_encoded_image = base64.b64encode(image_bytes).decode('utf-8')
    return base64_encoded_image

### 1. Settings

In [209]:
settings = {
    'accID': '621348950418460672', # Shrinath
    'accID': '1313274493921132544', # TMS US
    'tStart':  datetime(2025,7,8,23),
    'tEnd': datetime(2025,7,9,20),
    'region': 'India',
    'region': 'US',
    'vehID': None,
    'procDir': './AutoApprovalProc/',
}
settings['fetchAlertTypes'] = 'mobile_phone,seatbelt_violation'

vidType = {
    'mobile_phone': 'driver',
    'seatbelt_violation': 'driver',
}

settings['urlFetchAlerts'] = 'https://apis.intangles.com'
settings['urlUpdateAlerts'] = 'https://apis.intangles.com/alertlog/{alertID}/updateV2?&acc_id={accID}&token={token}'

settings['token'] = 'JBRHjp1tPFdyZwvRGblGwi-hIv5OmYu-cr--qzRE9rSCY6F1M5vxQt5Y7Wn9g7ur'  
if settings['region'].lower()=='us':
    urlUS = 'https://apis.intangles-aws-us-east-1.intangles.us'
    tokenUS = 'FCNlIrz1dHlHyd_QYxF9V8bmLLkxNm_IzA3vq73MXdhpqOcXPf1YGBNIlgnYvrRI'
    settings['urlFetchAlerts'] = urlUS
    settings['urlUpdateAlerts'] = 'https://apis.intangles-aws-us-east-1.intangles.us/alertlog/{alertID}/updateV2?&acc_id={accID}&token={token}'
    settings['token'] = tokenUS
    uAws.initiate_s3_session(isUS=True)


In [210]:
    
info = {
    'start': datetime.now(),
    'settings': settings,
}

### 2. Fetch Alerts & Apply Filters

In [211]:
t1 = time.time()
dAlerts, info = iApis.getVTAlerts(st = settings['tStart'],
                            en = settings['tEnd'],
                            baseUrl = settings['urlFetchAlerts'],
                            utoken = settings['token'],
                            alertType= settings['fetchAlertTypes'],
                           )
t2 = time.time()
info['t_fetchAlerts'] = t2-t1
info['n_alerts'] = len(dAlerts)
print("#Alerts=%d, took %d sec"%(len(dAlerts), t2-t1))



#Alerts=7, took 1 sec


In [212]:
from collections import Counter
Counter([x['type'] for x in dAlerts])

Counter({'seatbelt_violation': 4, 'mobile_phone': 3})

In [213]:
alertsToProcess = []
info['alertsToProcess'] = []
info['alertsDropped'] = []
for ent in dAlerts:
    # filter to keep entries in vehID only (if given)
    vid = ent['vehicle_id']
    if settings['vehID'] is not None:
        if vid not in settings['vehID']:
            continue
        
    # check if video is available
    typ = ent['type']
    vidUrl = None
    for url in ent.get('alert_values',{}).get('artefacts',[]):
        if url.find(vidType[typ]+'.mp4')>=0:
            vidUrl = url
    if vidUrl is None:
        info['alertsDropped'].append((ent['id'], typ, 'no video file'))
    else:
        info['alertsToProcess'].append((ent['id'], typ, vidUrl))
print('#dropped=',len(info['alertsDropped']))
print('#for process=',len(info['alertsToProcess']))

#dropped= 1
#for process= 6


In [208]:
dAlerts

[{'id': '1313375417322700800-mobile_phone-1751997744174',
  'location': {'latitude': 30.70404511, 'longitude': -111.8216293},
  'timestamp': 1751997744174,
  'show_user': False,
  'vehicle_id': '1313375417322700800',
  'account_id': '1313274493921132544',
  'address': 'carretera inernacional col, La Otra Banda, 83755 Altar, Son., Mexico',
  'type': 'mobile_phone',
  'tags': [],
  'driver_ids': ['1434690984804876295'],
  'active_driver': '1434690984804876295',
  'last_updated_at': 1752034321005,
  'is_alert_shared': False,
  'is_alert_marked_seen': False,
  'alert_values': {'alert_class': 'CLASS_NAMES_MOBILE',
   'detected_driver': '1436593489042210816',
   'detected_driver_type': 'unverified_drivers',
   'duration': 6.047,
   'end_time': 1751997749221,
   'start_time': 1751997744174,
   'artefacts': ['vod/1313375417322700800/2025-07-08_18-02-24-2025-07-08_18-02-25_road.mp4',
    'vod/1313375417322700800/2025-07-08_18-02-24-2025-07-08_18-02-25_driver.mp4'],
   'artefacts_available': Tru

### Get Video from S3

In [170]:
videoDir = settings['procDir']+'/vidsUS/'
os.makedirs(localDir, exist_ok=True)

files = []
for (aId, typ, vidUrl) in info['alertsToProcess']:
    files.append(vidUrl)
    if len(files)>40:
        break
    
t1 = time.time()
uAws.downloadFiles(files, outDirPath=videoDir)
t2 = time.time()
info['t_AWS'] = t2-t1

### Process each video via Dino

In [159]:
# initialize
import utilsAutoApprovalDino as uDino
reload(uDino)
uDino.init('AutoApprovalProc/assets/refData.json')

loading DINO model ...


Using cache found in C:\Users\Vikram.Melapudi/.cache\torch\hub\facebookresearch_dinov2_main


loading reference dataset ...
mobile_phone (137, 768)
seatbelt_violation (235, 768)
DONE.


In [173]:
counts = {'Approve':0, 'Ignore':0, 'Verify': 0}
processedAlerts = {}

In [174]:
tempDir = os.path.join(settings['procDir'],'temp/')
os.makedirs(tempDir, exist_ok=True)
    
nVid = 0
for (aId, typ, vidUrl) in info['alertsToProcess']:
    if aId in processedAlerts:
        continue
    nVid += 1
    print('process %d/%d...'%(nVid,len(info['alertsToProcess'])))
    t1 = time.time()
    localVidFile = uAws.s3ToLocalPath(vidUrl)
    vid = imageio.get_reader( os.path.join(videoDir,localVidFile) )
    N = 10 # vid.count_frames()
    img = vid.get_data(N//2)
    if img.shape[1]>800:
        if settings['region'].lower()=='us':
            img = img[:,img.shape[1]//2:]
        else:
            img = img[:,:img.shape[1]//2]

    qobj = uDino.procImage(img, saveNpz=False)#, forceLoad=True)
    dists, indxs = qobj['knn'].kneighbors(uDino.feats[typ], 3)

    xyDists = []
    for n in range(len(dists)):
        for m in range(3):
            x, y = uDino.index2xy(indxs[n,m], qobj)
            xyDists.append((x,y,dists[n,m]))
    xyDists = np.array(xyDists) 
    rules = uDino.refDataset.get(typ,{}).get('rules',{})
    # indx = np.where(xyDists[:,2]<rules['threshold'])[0]

    t2 = time.time()

    Nthresh = rules['passThreshold']
    Ntot = len(xyDists)
    Npass = np.sum(xyDists[:,2]<Nthresh)
    if Npass > Ntot//2+int(Ntot*rules['verifyPercentage']):
        Nres = rules["passAction"]
    elif Npass < Ntot//2-int(Ntot*0.1):
        Nres = rules["failAction"]
    else:
        Nres = 'Verify'

    print(qImageFile, typ, Ntot, Npass, Nres, '(%d s)'%(t2-t1))
    counts[Nres] += 1

    rimg = uDino.cv2.resize(img, (150,100))
    
    processedAlerts[str(aId)] = {
        'id': str(aId),
        'img': getBase64Image(rimg),
        'type': str(typ),
        'result': Nres,
        'Npass': Npass,
        'Ntot': Ntot,
        'runS': (t2-t1),
        'date': datetime.now()
    }

    if False:
        plt.figure(figsize=(7,3))
        plt.subplot(121)
        plt.imshow(qobj['image'])
    #     plt.plot(xyDists[indx,0], xyDists[indx,1], 'rx')
        plt.subplot(122)
        plt.plot(sorted(xyDists[:,2]))
        plt.hlines(45,0,len(xyDists))#,'r-')
        plt.xlabel('feat-index')
        plt.ylabel('NN-dist')
        plt.tight_layout()
        plt.show()
print('')
print('DONE.')

process 1/30...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 145 Ignore
process 2/30...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 227 Verify
process 3/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 659 Ignore
process 4/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 562 Ignore
process 5/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 691 Ignore
process 6/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 676 Ignore
process 7/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 642 Ignore
process 8/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 688 Ignore
process 9/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 686 Ignore
process 10/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 643 Ignore
process 11/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 687 Ignore
process 12/30...
./AutoApprovalProc/temp/tmp.png seatbelt_violation 705 684 Ignore
process 13/30...
./AutoAp

### Write a Summary File

In [175]:
with open(settings['procDir']+'/summaryUS.html','w') as f:
    f.write('''<html><head>
    
    <style>
    *{
      font-family: Arial;
      font-size: 12px;
      }
    th, td {
      padding-left: 10px;
      padding-right: 10px;
    }
    .approve {
        background-color:#DDFFDD;
        padding:5px;
        border-radius:8px;
    }
    .ignore {
        background-color:#FFDDDD;
        padding:5px;
        border-radius:8px;
    }
    .verify {
        background-color:#FFF955;
        padding:5px;
        border-radius:8px;
    }
    </style>
    <script>
    var settingsJSON = '%s'
    function clicked(el) {
      var els = document.getElementsByClassName(el.id);
      console.info(el.id+','+els.length);
      var dval = 'none';
      if(el.checked) dval = '';
      for(var n=0; n<els.length; n++) {
        if(els[n].tagName=='LABEL') continue;
        els[n].style.display = dval;
      }
    }
    </script>
    </head>
    <body style="text-align:center;">
    <h2 style="font-size:24px;background-color:#BBBBFF; padding:5px;">Auto Approval Review</h2>
    <div style="margin:10px"> accID=%s, vIDs=%s, time=[%s, %s]
    </div>
    <div style="margin:10px">
    <input type="checkbox" id="approve" onclick="clicked(this)"><label for="approve" id="lapprove" class="approve">approve</label>
    <input type="checkbox" id="ignore" onclick="clicked(this)"><label for="ignore" id="lignore" class="ignore">ignore</label>
    <input type="checkbox" id="verify" onclick="clicked(this)"><label for="verify" id="lverify" class="verify">verify</label>
    </div>
    <table style="margin:5px; margin-left:auto; margin-right:auto;">
    '''%(json.dumps(settings,default=str), 
         settings['accID'], str(settings['vehID']), 
         str(settings['tStart']), str(settings['tEnd'])) )
    
    header = ['Alert ID','Image','Type','Result','Score','Timing','Date']
    f.write('<tr>')
    for head in header:
        f.write('<th>%s</th>'%head)
    f.write('</tr>\n')
    
    for idd in processedAlerts:
        lst = processedAlerts[idd]
        f.write('<tr class="%s">'%lst['result'])
        f.write('<td>'+lst['id']+'</td>')
        f.write('<td><img src="data:image/png;base64,%s"></td>'%lst['img'])
        f.write('<td>'+lst['type']+'</td>')
        f.write('<td>%s</td>'%lst['result'])
        f.write('<td>%d/%d</td>'%(lst['Npass'], lst['Ntot']))
        f.write('<td>%1.2f</td> <td>%d</td>'%(lst['runS'], lst['date'].timestamp()))
        f.write('</tr>\n')
    
    f.write('''</table>
    <script>
    var el;
    el = document.getElementById("lapprove"); el.innerHTML = "Approve (%d)"; 
    el = document.getElementById("lignore"); el.innerHTML = "Ignore (%d)"; 
    el = document.getElementById("lverify"); el.innerHTML = "Verify (%d)";
    el.click();
    var lst = ['approve','ignore','verify'];
    for(var n=0; n<3; n++) clicked(document.getElementById(lst[n]));
    </script>
    </body></html>
    '''%(counts['Approve'],counts['Ignore'],counts['Verify'])
    )
    
      

### Update alerts

In [190]:
import requests

userShow = {
    'mobile_phone': {
        'Approve': (True, json.dumps({"approval_status":"approve", "remarks":"auto approval"})),
        'Ignore': (False, json.dumps({"approval_status":"decline", "remarks":"auto approval"})),
    },
    'seatbelt_violation': {
        'Approve': (True, json.dumps({"approval_status":"approve", "remarks":"auto approval"})),
        'Ignore': (False, json.dumps({"approval_status":"decline", "remarks":"auto approval"})),
    },
}


# ignoreData = {"user_show":False, 
#               "alert_values": json.dumps({"approval_status":"decline", "remarks":"Detection Issue: No yawn/No Eye Close"})
#              }

if False:
    alertUpdateURL = settings['urlUpdateAlerts']
    for aId in processedAlerts:
        ent = processedAlerts[aId]
        if ent['result']=='Verify':
            continue

        accId = settings['accID']
        token = settings['token']
        flag, values = userShow.get(ent['type']).get(ent['result'])

        ignoreData = {'user_show': flag, 'alert_values': values}
        print(aId, typ, ent['result'], flag)# , values)
        r = requests.post(alertUpdateURL.format(alertID=aId,accID=accId,token=token), ignoreData)
        rjd = r.json()


1313375417322700800-mobile_phone-1751741816603 seatbelt_violation Ignore False
1313375417322700800-seatbelt_violation-1751741488488 seatbelt_violation Ignore False
1380918636398510080-seatbelt_violation-1751742723359 seatbelt_violation Ignore False
1313375417322700800-seatbelt_violation-1751744158784 seatbelt_violation Ignore False
1313375417322700800-seatbelt_violation-1751745876782 seatbelt_violation Ignore False
1380918636398510080-seatbelt_violation-1751746501695 seatbelt_violation Ignore False
1313375417322700800-seatbelt_violation-1751747856345 seatbelt_violation Ignore False
1313375417322700800-seatbelt_violation-1751750993450 seatbelt_violation Ignore False
1380918636398510080-seatbelt_violation-1751751200510 seatbelt_violation Ignore False
1313375417322700800-seatbelt_violation-1751754262963 seatbelt_violation Ignore False
1313375417322700800-seatbelt_violation-1751755521098 seatbelt_violation Ignore False
1313375417322700800-seatbelt_violation-1751757515061 seatbelt_violation